In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col

spark = SparkSession.builder.appName('733').getOrCreate()
sc = spark.sparkContext

In [1]:
df = spark.read.csv('../quarterly_reports.csv', header=True).limit(20000)

# Number of total records
df.count()

# nullcounts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
# nullcounts.write.csv('nullcount')

20000

In [3]:
df.show()

+------+--------+------+----+---+------+------+------+-------+----+---------+--------------------+--------+-----+------+------+-----+-------+------+-------+-------+--------+--------+--------+------+----+---+----+----+------+----+---------+------+------+----+------+--------+----+----+-----+----+----+----------+----------+--------+----------+-----+----+----+----+----+------+------+----+------+--------+-----+----+-----+------+------+------+------+-------+----+----+--------+-------+--------+------+--------+------+----+--------+--------+-------+------+------+-----+------+------+------+-------+------+------+-----+----+------+------+-------+------+------+-------+------+------+------+----+-----+-----+-----+-------+-----+------+------+------+------+-------+------+------+-------+-------+------+------+-----+----+----+-------+--------+-------+--------+-------+----------+---------+-------+-----+----+-------+------+-------+------+---------+--------+------+----+------+----+--------+-------+------+---

In [4]:
df.shape()

AttributeError: 'DataFrame' object has no attribute 'shape'

In [40]:
df.select('tic').distinct().count()

12477

In [24]:
df.columns

NameError: name 'df' is not defined

In [41]:
misstatements = spark.read.csv('../misstatements.csv', header=True)

In [42]:
misstatements.count()

2309

In [43]:
misstatements.show()

+--------------------+------+--------+----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+
|            firmname|ticker|exchange|press_date|reason1|reason2|reason3|reason4|reason5|reason6|reason7|reason8|reason9|reason10|reason11|reason12|
+--------------------+------+--------+----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+
|System Software A...|  SSAX|  Nasdaq|01-07-1997|      0|      0|      0|      0|      0|      0|      0|      0|      1|       0|       0|       0|
|Soligen Technolog...|   SGT|    Amex|01-10-1997|      0|      0|      0|      0|      0|      0|      0|      0|      0|       1|       0|       0|
|Physicians Laser ...|  PHLZ|     OTC|01-16-1997|      1|      0|      0|      0|      0|      0|      0|      0|      0|       0|       0|       0|
|    Perceptron, Inc.|  PRCP|  Nasdaq|01-22-1997|      0|      0|      0|      0|      0|      0|      0| 

In [44]:
new_df = df.join(misstatements, df.tic == misstatements.ticker)

In [47]:
new_df.select('tic').distinct().count()

377

In [45]:
misstatements.where(misstatements.ticker == 'AE.2').show()

+--------+------+--------+----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+
|firmname|ticker|exchange|press_date|reason1|reason2|reason3|reason4|reason5|reason6|reason7|reason8|reason9|reason10|reason11|reason12|
+--------+------+--------+----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+
+--------+------+--------+----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+



In [46]:
selected_df = new_df.select(['tic', 'txtq_fn1', 'gvkey','datadate','fyearq','fqtr','fyr','indfmt','consol','popsrc','datafmt'])

In [10]:
selected_df.show()

+----+--------+------+--------+------+----+---+------+------+------+-------+
| tic|txtq_fn1| gvkey|datadate|fyearq|fqtr|fyr|indfmt|consol|popsrc|datafmt|
+----+--------+------+--------+------+----+---+------+------+------+-------+
|CECE|    null|001050|19810831|  1981|   1|  5|  INDL|     C|     D|    STD|
|CECE|    null|001050|19811130|  1981|   2|  5|  INDL|     C|     D|    STD|
|CECE|    null|001050|19820228|  1981|   3|  5|  INDL|     C|     D|    STD|
|CECE|    null|001050|19820531|  1981|   4|  5|  INDL|     C|     D|    STD|
|CECE|    null|001050|19820831|  1982|   1|  5|  INDL|     C|     D|    STD|
|CECE|    null|001050|19821130|  1982|   2|  5|  INDL|     C|     D|    STD|
|CECE|    null|001050|19830228|  1982|   3|  5|  INDL|     C|     D|    STD|
|CECE|    null|001050|19830531|  1982|   4|  5|  INDL|     C|     D|    STD|
|CECE|    null|001050|19830831|  1983|   1|  5|  INDL|     C|     D|    STD|
|CECE|    null|001050|19831130|  1983|   2|  5|  INDL|     C|     D|    STD|

In [11]:
selected_df.count()

3446

In [12]:
tic_column = selected_df['tic']

In [13]:
type(tic_column)

pyspark.sql.column.Column

In [14]:
from pyspark.sql.functions import col, countDistinct

freq= selected_df.agg((countDistinct(col('tic'))))

In [15]:
freq.show()

+-------------------+
|count(DISTINCT tic)|
+-------------------+
|                  9|
+-------------------+



In [7]:
annual_df = spark.read.csv('../annual_compustat.csv', header=True) #.limit(20000)

In [9]:
type(annual_df)

pyspark.sql.dataframe.DataFrame

In [13]:
annual_df.columns

['gvkey',
 'datadate',
 'fyear',
 'indfmt',
 'consol',
 'popsrc',
 'datafmt',
 'tic',
 'cusip',
 'conm',
 'acctchg',
 'acctstd',
 'acqmeth',
 'adrr',
 'ajex',
 'ajp',
 'bspr',
 'compst',
 'curcd',
 'curncd',
 'currtr',
 'curuscn',
 'final',
 'fyr',
 'ismod',
 'ltcm',
 'ogm',
 'pddur',
 'scf',
 'src',
 'stalt',
 'udpl',
 'upd',
 'apdedate',
 'fdate',
 'pdate',
 'acchg',
 'acco',
 'acdo',
 'aco',
 'acodo',
 'acominc',
 'acox',
 'acoxar',
 'acqao',
 'acqcshi',
 'acqgdwl',
 'acqic',
 'acqintan',
 'acqinvt',
 'acqlntal',
 'acqniintc',
 'acqppe',
 'acqsc',
 'act',
 'adpac',
 'aedi',
 'afudcc',
 'afudci',
 'aldo',
 'am',
 'amc',
 'amdc',
 'amgw',
 'ano',
 'ao',
 'aocidergl',
 'aociother',
 'aocipen',
 'aocisecgl',
 'aodo',
 'aol2',
 'aoloch',
 'aox',
 'ap',
 'apalch',
 'apb',
 'apc',
 'apofs',
 'aqa',
 'aqc',
 'aqd',
 'aqeps',
 'aqi',
 'aqp',
 'aqpl1',
 'aqs',
 'arb',
 'arc',
 'arce',
 'arced',
 'arceeps',
 'artfs',
 'at',
 'aul3',
 'autxr',
 'balr',
 'banlr',
 'bast',
 'bastr',
 'batr',
 'bc

In [15]:
'rea' in annual_df.columns

True

In [21]:
annual_df.select('rea').show()

+-------+
|    rea|
+-------+
|   null|
|   null|
|   null|
|   null|
|   null|
|   null|
|   null|
|   null|
| 2.7720|
| 0.0000|
| 0.0000|
| 0.0000|
| 0.0000|
| 0.0000|
|-1.6560|
| 0.0000|
| 0.0000|
|   null|
|   null|
|   null|
+-------+
only showing top 20 rows



In [31]:
annual_df.count()

505781

In [32]:
annual_mis = annual_df.join(misstatements, annual_df.tic == misstatements.ticker)

In [33]:
annual_mis.show()

+------+--------+-----+------+------+------+-------+----+---------+--------------------+-------+-------+-------+----+------+------+----+------+-----+------+------+-------+-----+---+-----+----+----+-----+----+---+-----+----+---+--------+-----+-----+------+----+----+------+-----+-------+------+------+-----+-------+-------+-----+--------+-------+--------+---------+------+-----+------+-----+----+------+------+----+------+----+----+----+----+------+---------+---------+-------+---------+----+----+-------+------+------+------+----+----+-----+----+-------+----+-----+------+----+-----+------+----+----+----+-----+-------+-----+------+----+-----+----+-----+------+-------+----+----+----+------+-----+-----+----+-----+----+-------+-----+----+-----+-----+-----+------+------+------+----+----+-----+----+-------+-------+-------+----+-----+----+-----+----+----+----+----+------+------+-------+----+----+-------+------+-------+-----+-------+-----+-------+-------+------+------+------+------+----+----+----+--

In [34]:
annual_mis.select('tic').distinct().count()

1095

In [35]:
annual_df.select('tic').distinct().count()

36349

In [37]:
len(annual_df.columns)

1778